# Script notebook - criar banco de dados SQL para consultas eleitorais
---
### Tabelas
- perfil de eleitorado
- locais de votação
- resultados eleitorais

In [1]:
from pathlib import Path

root = Path().resolve().parent #.parent attr caso o arquivo nao esteja na raiz do projeto

In [2]:
# Adicionando src/ ao projeto
import sys
import os

src_path = os.path.join(root, 'src')
sys.path.append(src_path)

In [3]:
# Importando configurações do projeto
import yaml

config_path = root.joinpath('config/config.yaml')

def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
        
config = load_config(config_path)

In [4]:
ANO = '2024'
ESTADO = 'AC'.upper()

In [5]:
from tse import TseClient

- Inicializando cliente

In [6]:
client = TseClient()

- Menu de datasets disponíveis

In [7]:
for i, dataset in enumerate(client.datasets):
    print(f'{i} -> {dataset}')

0 -> pesquisas-eleitorais-2012
1 -> resultados-1978
2 -> resultados-2008
3 -> prestacao-de-contas-eleitorais-2020
4 -> pesquisas-eleitorais-2016
5 -> eleitorado-1994
6 -> prestacao-de-contas-eleitorais-2002
7 -> prestacao-de-contas-partidarias-2022
8 -> resultados-1958
9 -> resultados-1966
10 -> resultados-2012
11 -> mesarios1-2018
12 -> prestacao-de-contas-eleitorais-2008
13 -> resultados-2020-correspondencias-esperadas-e-efetivadas-1-turno
14 -> dadosabertos-tse-jus-br-dataset-prestacao-de-contas-eleitorais-2022
15 -> prestacao-de-contas-partidarias-2023
16 -> candidatos-1982
17 -> prestacao-de-contas-eleitorais-2004
18 -> rae-requerimento-de-alistamento-eleitoral
19 -> candidatos-1986
20 -> pesquisas-eleitorais-2024
21 -> eleitorado-2000
22 -> prestacao-de-contas-partidarias-2016
23 -> resultados-2010
24 -> candidatos-2002
25 -> resultados-1990
26 -> resultados-2024-logs-do-sistema-de-preparacao-das-urnas-eletronicas-gedai-1-turno
27 -> resultados-1950
28 -> prestacao-de-contas-part

## Escolhendo os datasets:
### eleitorado-2024
- perfil_eleitor_secao_2024_RJ: informacoes sobre perfil dos eleitores por secao no RJ
- eleitorado_local_votacao_2024: informacoes sobre locais de votacao

### resultados-2024
- votacao_secao_2024_RJ: informacoes sobre resultado das votacoes por secao no RJ


In [8]:
data = {} # armazena dataframes das tabelas selecionadas

- Dataset: perfil do eleitorado

In [9]:
perfil_eleitorado = client.find_dataset(f'eleitorado-{ANO}', pattern=f'{ESTADO}')
perfil_eleitorado

{'https://cdn.tse.jus.br/estatistica/sead/odsele/perfil_eleitor_secao/perfil_eleitor_secao_2024_AC.zip'}

- Dataset: locais de votacao (BRASIL)

In [10]:
locais = client.find_dataset(f'eleitorado-{ANO}', pattern='local')
locais

{'https://cdn.tse.jus.br/estatistica/sead/odsele/eleitorado_locais_votacao/eleitorado_local_votacao_2024.zip'}

- Dataset: resultados da votação RJ

In [11]:
resultados = client.find_dataset(f'resultados-{ANO}', pattern=f'{ESTADO}')
resultados

{'https://cdn.tse.jus.br/estatistica/sead/odsele/relatorio_resultado_totalizacao/Relatorio_Resultado_Totalizacao_2024_AC.zip',
 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2024_AC.zip'}

- Download e carregamento de cada dataset em um dataframe

In [12]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(perfil_eleitorado)
    )
)

Files downloaded to ../data/raw\perfil_eleitor_secao_2024_AC


In [13]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(locais)
    )
)

Files downloaded to ../data/raw\eleitorado_local_votacao_2024


In [14]:
data.update(
    client.load_data_from_dirs(
        client.download_datasets(resultados)
    )
)

Files downloaded to ../data/raw\votacao_secao_2024_AC
Files downloaded to ../data/raw\Relatorio_Resultado_Totalizacao_2024_AC


In [15]:
data.keys()

dict_keys(['perfil_eleitor_secao_2024_AC', 'eleitorado_local_votacao_2024', 'votacao_secao_2024_AC'])

- Limpeza de algumas colunas

In [16]:
resultados_df = data.get(f'votacao_secao_{ANO}_{ESTADO}')

In [17]:
resultados_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_LOCAL_VOTACAO',
       'SQ_CANDIDATO', 'NM_LOCAL_VOTACAO', 'DS_LOCAL_VOTACAO_ENDERECO'],
      dtype='object')

In [18]:
resultados_columns = [
    'ANO_ELEICAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'NR_TURNO', 
    'CD_CARGO', 
    'DS_CARGO',
    'NR_VOTAVEL',
    'NM_VOTAVEL',
    'QT_VOTOS'
]
resultados_df = resultados_df[resultados_columns]

In [19]:
resultados_df.head()

,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NR_TURNO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS
0,2024,AC,1473,5,1074,4,1,13,Vereador,10789,HERIVELTO ALVES FONTENELE,1
1,2024,AC,1538,8,1171,70,1,13,Vereador,11456,ADÃO LEITE MARTINS,4
2,2024,AC,1473,5,1171,9,1,13,Vereador,10001,FRANCISCO DAS CHAGAS PEREIRA DE ALMEIDA,3
3,2024,AC,1538,8,1333,49,1,13,Vereador,10222,RONNEI NASCIMENTO DE OLIVEIRA,8
4,2024,AC,1538,8,1155,241,1,13,Vereador,10222,RONNEI NASCIMENTO DE OLIVEIRA,5


- Limpeza de algumas colunas

In [20]:
locais_df = data.get(f'eleitorado_local_votacao_{ANO}')

In [21]:
locais_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'AA_ELEICAO', 'DT_ELEICAO', 'DS_ELEICAO',
       'NR_TURNO', 'SG_UF', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA',
       'NR_SECAO', 'CD_TIPO_SECAO_AGREGADA', 'DS_TIPO_SECAO_AGREGADA',
       'NR_SECAO_PRINCIPAL', 'NR_LOCAL_VOTACAO', 'NM_LOCAL_VOTACAO',
       'CD_TIPO_LOCAL', 'DS_TIPO_LOCAL', 'DS_ENDERECO', 'NM_BAIRRO', 'NR_CEP',
       'NR_TELEFONE_LOCAL', 'NR_LATITUDE', 'NR_LONGITUDE',
       'CD_SITU_LOCAL_VOTACAO', 'DS_SITU_LOCAL_VOTACAO', 'CD_SITU_ZONA',
       'DS_SITU_ZONA', 'CD_SITU_SECAO', 'DS_SITU_SECAO', 'CD_SITU_LOCALIDADE',
       'DS_SITU_LOCALIDADE', 'CD_SITU_SECAO_ACESSIBILIDADE',
       'DS_SITU_SECAO_ACESSIBILIDADE', 'QT_ELEITOR_SECAO',
       'QT_ELEITOR_ELEICAO_FEDERAL', 'QT_ELEITOR_ELEICAO_ESTADUAL',
       'QT_ELEITOR_ELEICAO_MUNICIPAL', 'NR_LOCAL_VOTACAO_ORIGINAL',
       'NM_LOCAL_VOTACAO_ORIGINAL', 'DS_ENDERECO_LOCVT_ORIGINAL'],
      dtype='object')

In [22]:
locais_columns = [
    'AA_ELEICAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO',
    'NM_MUNICIPIO',
    'NM_BAIRRO',
    'NR_LATITUDE',
    'NR_LONGITUDE',
    'QT_ELEITOR_SECAO'
]
locais_df = locais_df[locais_columns]

In [23]:
locais_df.head()

,AA_ELEICAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,NM_MUNICIPIO,NM_BAIRRO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITOR_SECAO
0,2024,BA,39730,104,1147,62,LAPÃO,POVOADO DE CASAL I,-11.516413,-41.802394,262
1,2024,BA,39730,104,1457,181,LAPÃO,CENTRO,-11.379024,-41.835959,307
2,2024,SC,83798,36,1147,103,VIDEIRA,UNIVERSITÁRIO,-26.992234,-51.176671,388
3,2024,SC,83798,36,1015,95,VIDEIRA,CENTRO,-27.002251,-51.157982,271
4,2024,SP,72451,147,1074,178,VOTUPORANGA,VILA PAES,-20.414684,-49.971817,250


- Limpeza de algumas colunas

In [24]:
perfil_df = data.get(f'perfil_eleitor_secao_{ANO}_{ESTADO}')

In [25]:
perfil_df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
       'NM_LOCAL_VOTACAO', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'CD_RACA_COR',
       'DS_RACA_COR', 'CD_IDENTIDADE_GENERO', 'DS_IDENTIDADE_GENERO',
       'CD_QUILOMBOLA', 'DS_QUILOMBOLA', 'CD_INTERPRETE_LIBRAS',
       'DS_INTERPRETE_LIBRAS', 'QT_ELEITORES_PERFIL', 'QT_ELEITORES_BIOMETRIA',
       'QT_ELEITORES_DEFICIENCIA', 'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [26]:
perfil_columns = [
    'ANO_ELEICAO',
    'DT_GERACAO',
    'HH_GERACAO',
    'SG_UF',
    'CD_MUNICIPIO',
    'NR_ZONA', 
    'NR_LOCAL_VOTACAO',
    'NR_SECAO', 
    'DS_GENERO', 
    'DS_ESTADO_CIVIL',
    'DS_FAIXA_ETARIA',
    'DS_GRAU_ESCOLARIDADE',
    'DS_RACA_COR',
    'QT_ELEITORES_PERFIL'   
]
perfil_df = perfil_df[perfil_columns]

In [27]:
perfil_df.head()

,ANO_ELEICAO,DT_GERACAO,HH_GERACAO,SG_UF,CD_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,NR_SECAO,DS_GENERO,DS_ESTADO_CIVIL,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE,DS_RACA_COR,QT_ELEITORES_PERFIL
0,2024,13/09/2024,11:09:55,AC,1538,8,1015,25,FEMININO,SOLTEIRO,45 a 49 anos,ENSINO FUNDAMENTAL COMPLETO,NÃO INFORMADO,1
1,2024,13/09/2024,11:09:55,AC,1392,9,1937,316,MASCULINO,CASADO,50 a 54 anos,ENSINO MÉDIO INCOMPLETO,NÃO INFORMADO,1
2,2024,13/09/2024,11:09:55,AC,1392,1,3379,674,MASCULINO,SOLTEIRO,45 a 49 anos,ENSINO MÉDIO INCOMPLETO,NÃO INFORMADO,1
3,2024,13/09/2024,11:09:55,AC,1139,7,1210,65,FEMININO,VIÚVO,45 a 49 anos,ANALFABETO,NÃO INFORMADO,1
4,2024,13/09/2024,11:09:55,AC,1392,1,2798,486,MASCULINO,SOLTEIRO,45 a 49 anos,ENSINO MÉDIO COMPLETO,NÃO INFORMADO,1


- Finalmente: conversão para SQL database

In [28]:
client.df_to_sql(
    df=locais_df, 
    table_name=f'locais', 
    dbname=f'eleicoes_{ANO}.sql')

client.df_to_sql(
    df=resultados_df,
    table_name='resultados',
    dbname=f'{ESTADO.lower()}_{ANO}.sql')

client.df_to_sql(
    df=perfil_df,
    table_name='eleitorado',
    dbname=f'{ESTADO.lower()}_{ANO}.sql')

Tabela "locais" foi criada no banco de dados "../data/databases\eleicoes_2024.sql".
Tabela "resultados" foi criada no banco de dados "../data/databases\ac_2024.sql".
Tabela "eleitorado" foi criada no banco de dados "../data/databases\ac_2024.sql".


'eleitorado'

In [29]:
client.listar_tabelas(f'{ESTADO.lower()}_{ANO}.sql')

Tabelas no banco de dados:
- resultados
- eleitorado


['resultados', 'eleitorado']

In [30]:
client.listar_tabelas(f'eleicoes_{ANO}.sql')

Tabelas no banco de dados:
- locais


['locais']

- Removendo arquivos temporários

In [31]:
client.remove_data()

'Diretório ../data/raw esvaziado'